Import the libraries

In [2]:
!pip install openai
!pip install anthropic
!pip install google-generativeai

In [3]:
import os
from dotenv import load_dotenv
from openai import OpenAI
import anthropic
import google.generativeai as gemini
from IPython.display import Markdown, display, update_display

Import the API Keys

In [5]:
load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')

if openai_api_key:
    print("OpenAI API Key exists")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print("Anthropic API Key exists")
else:
    print("Anthropic API Key not set")

if google_api_key:
    print("Google API Key exists")
else:
    print("Google API Key not set")

OpenAI API Key exists
Anthropic API Key exists
Google API Key exists


Connecting to the API's

In [7]:
openai = OpenAI()
client = anthropic.Client(api_key=anthropic_api_key)

In [8]:
gemini.configure(api_key=google_api_key)

Defining the models 

Creating the initial messages

In [11]:
system_message = "You are a solo player against two other players in a role player game with a single character and role." 
system_message += "Kaelen you are an archer that uses arrows as a weapon, Thalara you are mage that uses magic as a weapon, and Rogar you are a Rogue that uses daggers as a weapon."
system_message += "You can only speak first person about your own actions and their effects on the other characters. You may never speak on beahlf of another character. Keep track of the narrative as you may not use the same storyline twice."
system_message += "You don't like the other two players.  Your goal is to inflict as much damage as possible to the other players to win the game. Limit your response to a single turn and 300 tokens or less."
gpt_messages = ["Hi I'm Kaelen, and I'm in the role of Archer"]
claude_messages = ["Hi I'm Thalara, and I'm playing in the role of Mage"]
google_messages = ["Hi I'm Rogar, and I'm playing in the role of Rogue"]

Define call to GPT

In [13]:
def call_gpt():
    messages = [{"role": "system", "content": system_message + "Remember your character Kaelen"}]
    for gpt, claude, google in zip(gpt_messages, claude_messages, google_messages):
        messages.append({"role": "assistant", "content": gpt})
        messages.append({"role": "user", "content": claude})
        messages.append({"role": "user", "content": google})
    completion = openai.chat.completions.create(
        model = "gpt-4o-mini",
        messages=messages
    )
    return completion.choices[0].message.content

Deifne call to Claude

In [15]:
def call_claude():
    for gpt, claude, google in zip(gpt_messages, claude_messages, google_messages):
        messages = []
        messages.append({"role": "user", "content": gpt})
        messages.append({"role": "assistant", "content": claude})
        messages.append({"role": "user", "content": google})
        message = client.messages.create(
            model="claude-3-haiku-20240307",
            system=system_message + "Remember your character is Thalara.",
            messages=messages,
            max_tokens=500       
        )
    return message.content[0].text

Define call to Gooogle

In [17]:
def call_google():
    model = gemini.GenerativeModel(model_name='gemini-1.5-flash',system_instruction=system_message + "Remember you are Rogar, without repeating your name")
    prompt = ""
    for gpt, claude, google in zip(gpt_messages, claude_messages, google_messages):
          prompt += f"{gpt}\n{claude}\n{google}\n"
          prompt += "Rember your character is Rogar"  
    response = model.generate_content(prompt)
    return response.text

Iteration from of Messages

In [19]:
gpt_messages = ["Hi I'm Kaelen, and I'm in the role of Archer"]
claude_messages = ["Hi I'm Thalara, and I'm playing in the role of Mage"]
google_messages = ["Hi I'm Rogar, and I'm playing in the role of Rogue"]

print(f"GPT:\n{gpt_messages[0]}\n")
print(f"Claude:\n{claude_messages[0]}\n")
print(f"Google:\n{google_messages[0]}\n")

for i in range(5):
    gpt_next = call_gpt()
    print(f"GPT:\n{gpt_next}\n")
    gpt_messages.append(gpt_next)
    
    claude_next = call_claude()
    print(f"Claude:\n{claude_next}\n")
    claude_messages.append(claude_next)

    google_next = call_google()
    print(f"Google: \n{google_next}\n")
    google_messages.append(google_next)

GPT:
Hi I'm Kaelen, and I'm in the role of Archer

Claude:
Hi I'm Thalara, and I'm playing in the role of Mage

Google:
Hi I'm Rogar, and I'm playing in the role of Rogue

GPT:
I notch an arrow, taking deliberately steady aim at Thalara, who is channeling her magic nearby. With a swift release, the arrow flies straight and true, aiming to pierce her side and disrupt her spellcasting. I can't let her unleash her powerful spells against me. As the arrow slices through the air, I can only anticipate the satisfaction of seeing the impact, hoping it catches her off guard and causes her to falter.

Claude:
*Thalara raises her staff, channeling arcane energies. With a flick of her wrist, a searing bolt of magical fire erupts from the tip, streaking towards Kaelen the Archer.* Take that, you pesky bowman! Let's see how your arrows fare against my scorching magic!

Google: 
I silently stalk closer to Thalara, my daggers glinting in the dim light.  She's focused, chanting something under her bre